In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from tir_lib import *
from globals import *
import time
import importlib
import pandas as pd
import math
from tqdm import tqdm
import json
import sys
from utils import *

################## CONFIG #######
input_json = "AFTER_ROUND_0_RESPONSES_20k_DS_1.json"
target_TIR_json = "TIR_FULL_RESPONSES_ROUND_1_20k_DS_1.json"
not_worked_json = "AFTER_ROUND_1_RESPONSES_20k_DS_1.json"

def read_json_1(input_json):
    with open(input_json, "r") as f:
        data = json.load(f)
        return data
        
TIR_FULL_RESPONSES = []
AFTER_ROUND_RESPONSES = []
#################################
data_list = read_json_1(input_json)
N = len(data_list)
print(f"samples = {N}")
all_q = [item["q"] for item in data_list]
all_r = [item["r"] for item in data_list]
all_a = [item["a"] for item in data_list]

/opt/conda/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


samples = 7974


In [2]:
#################################
def get_tir(q,r,a):
    length_response = get_length(r)
    b = True
    code_generated_freq = r.count("```python")
    output_generated_freq = r.count("```output")
    ########### IF CODE IS PRESENT ######################
    if code_generated_freq == output_generated_freq + 1:
#         resp = get_code_completion(q, r)
        new_r = r + "```"
        resp_with_output, real_tir = add_output_at_end(new_r)
        added_response  = get_extra_completion(q, resp_with_output)
        full_response = resp_with_output + added_response
    elif code_generated_freq > 0:
            added_r = insert_hint(q,r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
    ########### IF CODE IS NOT PRESENT ######################
    elif code_generated_freq == 0:
        choice = random.choices([1,0], weights = [0.1, 0.9], k = 1)[0]
        if choice or length_response < 3200:
            added_r = add_prompt_at_end(r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
        if length_response > 3200 - 1:
            added_r = insert_hint(q,r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
    else:
        raise Exception("some issue with code if - else") 
        
    gen_ans = extract_boxed_texts(full_response)
    if gen_ans is None:
        gen_ans = get_answer(full_response)
    if real_tir and (gen_ans == str(a) or int(verify_correctness(gen_ans, a))):
        TIR_FULL_RESPONSES.append({"q": q, "r":full_response, "a":a})
        print("TIR succeeded !")
    else:
        AFTER_ROUND_RESPONSES.append({"q": q, "r":full_response, "a":a})
        print("TIR failed...")
        b = False
    return b

In [3]:
len(TIR_FULL_RESPONSES)

0

In [ ]:
%%time
# math.ceil(len(data_list)/200)
for i in tqdm(range(math.ceil(len(data_list)/150))):
    start = 150*i
    end = 150*(i+1) 
    return_val = parallelize(get_tir, all_q[start:end] , all_r[start:end], all_a[start:end])
    print(return_val)

  0%|          | 0/54 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (17974 > 16384). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (16441 > 16384). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (18252 > 16384). Running this sequence through the model will result in indexing errors


api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error oc

2025-03-27 19:41:56.710700: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 19:41:56.753749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


api error occured!


2025-03-27 19:41:57.584446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error oc

  6%|▌         | 3/54 [04:33<1:17:03, 90.67s/it]

TIR failed...
[False, 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', False, 'can only concatenate str (not "tuple") to str', False, False, 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', False, 'can only concatenate str (not "tuple") to str', False, False, False, 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', False, 'can only concatenate str (not "tuple") to str', False, 'can only concatenate str (not "tuple") to str', False, 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str (not "tuple") to str', 'can only concatenate str 

Process Process-501:
Process Process-451:
Process Process-512:
Process Process-506:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.11/site-packages/sympy/core/assumptions.py", line 499, in getit
    return self._assumptions[fact]
           ~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
KeyError: 'zero'
  File "/opt/conda/lib/python3.11/mu

TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...


In [13]:
6

6

In [11]:
print(return_val)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, Fal

In [5]:
def save_unique(dict_list, name = "xx.json"):
    json_unique = []
    st = set()
    for item in dict_list:
        if item["q"] in st:
            pass
        else:
            st.add(item["q"])
            json_unique.append(item)
    print(f"final length: {len(json_unique)}")
    with open(name, "w") as f:
        json.dump(json_unique, f , indent = 4)
        
save_unique(TIR_FULL_RESPONSES, target_TIR_json)
save_unique(AFTER_ROUND_RESPONSES, not_worked_json)

final length: 1328
final length: 7974
